# Exploratory data analysis for Neighborhoods in Auckland
**A Capstone Project - The Battle of Neighborhoods**


## Introduction

Auckland(Tāmaki Makaurau) is the biggest city in New Zealand, with the population growing, the crimes happening frequency is increasing too. 
This project is to help people who migrate to Auckland to know more about the neighborhoods in Auckland.

### Current Problems
In Auckland, neighborhood and neighborhood are very different, some neighborhood is quite, some are busy. 
With the population growing, the crime frequency and safety of neighborhood are changing too. 
For example, Albany was the place for trash land filling, but now more and more houses are built, at the same time land price increased too.
New Zealand Police published the crime dataset on its website, but the problem there is no easy and direct way to know about the neighborhoods' safety.
This project is to help to solve this problem to let the residents and new migrants in Auckland to know about their neighborhoods.

### Dataset Required

New Zealand goverment provides crime data which is public, to build this project, there are 2 data sets required.
- `NZ Crimes Data`
- `Neighborhood Coordinate Data`

The crime dataset can be easily gotten from NZ Police Website, but they do not offer a neighborhood and coordinate dataset.
To get the coordinate data, we can use `geopy`'s Nominatim function to call Open Street Map geocode api. 
There might be a problem to use this method that there is a limit for the api usage. In this project, we generated a JSON file to build a dictionary of neighborhood and coordinates.

#### Neighborhood Data Preparation
The first data can be easily got through `police.govt.nz`. There is more than 400 neighborhood in Auckland. It is easy to get time out error when getting location through `geopy` geocoder. To fix this problem, the `locations_2020.sjon` file is used. This json file contains all location data required for this project.

To generate this file, each API calling of the geocoder is waited by 1 or 2 seconds, and every about 60 to 90 api calls are run separatelly and manuelly.


## References
- [Police Stat](https://www.police.govt.nz/about-us/publications-statistics/data-and-statistics)
- [Foursquare API](https://developer.foursquare.com/)
- [geopy](https://www.police.govt.nz/about-us/publications-statistics/data-and-statistics)

## Table of Contents
For Notebook Implementation

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Data</a>

2. <a href="#item2">Clustering</a>

3. <a href="#item3">ETA Exploratory data analysi</a>
   
</font>
</div>

## 1 Data Preparation

### 1.1 Prepare Python Dependecies

In [1]:
# instead of use request to call foursquare api, use this python client
!pip install foursquare

In [2]:
# other packages required here
!conda install -c conda-forge geopy matplotlib python-dotenv folium=0.5.0 --yes 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.5.0
    - geopy
    - matplotlib
    - python-dotenv


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    pyqt-5.6.0                 |py36h13b7fb3_1008         5.4 MB  conda-forge
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    sip-4.18.1                 |py36hf484d3e_1000         277 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    python-doten

### 1.2 Prepare Crimes Data

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('./NZ_Crimes.csv')

In [5]:
df_akl = df[df['Territorial Authority']=='Auckland.']

columns=[
    'ANZSOC Division',
    'ANZSOC Group',
    'ANZSOC Subdivision',
    'Area Unit',
    'Locn Type Division',
    'Meshblock',
    'Occurrence Day Of Week',
    'Occurrence Hour Of Day',
    'Weapon',
    ' Month Year',
]

df_akl=df_akl[columns].astype({
    'Meshblock': 'int32',
    'Occurrence Hour Of Day': 'int32',
})

# formate the date (Month Year), using date instead of string
from datetime import datetime
df_akl[' Month Year']=df_akl[' Month Year'].apply(
    lambda x: datetime.strptime(x, '%B %Y')
)

df_akl = df_akl.rename(columns={
    ' Month Year': 'YearMonth'
})

df_akl.head()

,ANZSOC Division,ANZSOC Group,ANZSOC Subdivision,Area Unit,Locn Type Division,Meshblock,Occurrence Day Of Week,Occurrence Hour Of Day,Weapon,YearMonth
24,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Albany.,Other Location,178900,Saturday,15,Not Applicable,2019-01-01
25,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Auckland Central West.,Other Location,433501,Saturday,15,Not Applicable,2019-01-01
26,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Auckland Central West.,Other Location,433600,Saturday,15,Not Applicable,2019-01-01
27,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Auckland Central West.,Other Location,434000,Saturday,15,Not Applicable,2019-01-01
28,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Mt Wellington South.,Other Location,637400,Saturday,15,Not Applicable,2019-01-01


In [6]:
df_akl.dtypes

ANZSOC Division                   object
ANZSOC Group                      object
ANZSOC Subdivision                object
Area Unit                         object
Locn Type Division                object
Meshblock                          int32
Occurrence Day Of Week            object
Occurrence Hour Of Day             int32
Weapon                            object
YearMonth                 datetime64[ns]
dtype: object

In [7]:
rows_num = df_akl.shape[0]
print('crimed date is loaded and prepared, totally {} rows'.format(rows_num))

crimed date is loaded and prepared, totally 74156 rows


### 1.2 Prepare Location Data

the way to get the latitude and longitude for each suburbs (area names)
```
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="akl_spider")
location = geolocator.geocode("{} auckland new zealand".format('albany'))
```
the way generated the `area_location.json` notebook.
```
with open('area_location.json', 'w') as file:
    json.dump(area_dict, file)
```

loading the generated json file instead of run the geocode each time will increase notebook speadd siginifitantly.


In [8]:
import json
areas = list(df_akl['Area Unit'].unique())
# this json file contains suburbs of Auckland
## which is generated through calling geopy geocode function
## it is too heavy to call it every time to run the notebook, so the json file is generated
# some one which cannot get through 'geocoder' which will be set to (0,0)
# for example `Inlet-Waitemata Harbour` which is not a correct suburb in Auckland, this kind of rows will be droped in this step

locations_2020 = {}
with open('locations_2020.json', 'r') as file:
    locations_2020=json.load(file)

In [9]:
if (len(locations_2020.keys()) < len(areas)):
    # the json file loades seems like outdated
    from geopy.geocoders import Nominatim
    import time
    locations = []

    def get_location(area_name, ua="Mozilla/5.0 (X11; Linux i686; rv:67.0) Gecko/20100101 Firefox/67.0"):
        geolocator = Nominatim(user_agent=ua)
        location = geolocator.geocode("{} auckland new zealand".format(area_name))
        time.sleep(1)
        return location

    def get_locations(area_names):
        return list(map(get_location, area_names))

    # runing this too fast will cause time out error
    # better to run the codes below in separate cells manuelly
    locations += get_locations(areas[:50])
    locations += get_locations(areas[50:100])
    locations += get_locations(areas[100:180])
    locations += get_locations(areas[180:260])
    locations += get_locations(areas[260:350])
    locations += get_locations(areas[350:400])
    locations += get_locations(areas[400:])
    locations_2020 = locations
else:
    print('locations is imported through loading json file')

locations is imported through loading json file


In [10]:
# a simple test to make sure the json is loaded correctly
print('json loading successfully ' if 'Albany.' in locations_2020.keys() and len(locations_2020.keys()) >= len(areas) else '') 

json loading successfully 


In [11]:
df_akl['latitude'] = df_akl['Area Unit'].apply(lambda x: locations_2020[x][0])
df_akl['longitude'] = df_akl['Area Unit'].apply(lambda x: locations_2020[x][1])

df_akl = df_akl[df_akl['latitude']!=0]

In [12]:
rows_num_after_drop = df_akl.shape[0]

In [13]:
df_akl.head()

,ANZSOC Division,ANZSOC Group,ANZSOC Subdivision,Area Unit,Locn Type Division,Meshblock,Occurrence Day Of Week,Occurrence Hour Of Day,Weapon,YearMonth,latitude,longitude
24,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Albany.,Other Location,178900,Saturday,15,Not Applicable,2019-01-01,-36.727058,174.698055
25,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Auckland Central West.,Other Location,433501,Saturday,15,Not Applicable,2019-01-01,-36.848911,174.765226
26,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Auckland Central West.,Other Location,433600,Saturday,15,Not Applicable,2019-01-01,-36.848911,174.765226
27,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Auckland Central West.,Other Location,434000,Saturday,15,Not Applicable,2019-01-01,-36.848911,174.765226
28,Theft and Related Offences,Theft From Retail Premises,Theft (Except Motor Vehicles),Mt Wellington South.,Other Location,637400,Saturday,15,Not Applicable,2019-01-01,-36.904793,174.835118


In [14]:
print('there {} rows which does not have a clean area name'.format(rows_num - rows_num_after_drop))
print('now we have {} rows in the table for the next step'.format(rows_num_after_drop))

there 4649 rows which does not have a clean area name
now we have 69507 rows in the table for the next step


### 1.3 Prepare Data For folium

In [15]:
# clean this area unit which can be found in geocode, but which is not a correct area name in Auckland
df_akl = df_akl[df_akl['Area Unit'] != '-29']

In [16]:
df_borough= df_akl.groupby('Area Unit').mean()
# df_borough['Crime Count']=df_akl.groupby('Area Unit').count()['YearMonth']

In [17]:
df_borough['CrimeCount'] = df_akl.groupby('Area Unit').count()['YearMonth']

In [18]:
df_borough=df_borough.drop(['Meshblock', 'Occurrence Hour Of Day'], axis=1).reset_index().rename(columns={'Area Unit': 'Neighborhood'})

## 2 Cluster neighborhood in Auckland by location

In [19]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 10

df_to_cluster = df_borough.drop(['Neighborhood','CrimeCount'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_to_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([9, 3, 6, 0, 0, 7, 9, 7, 3, 9], dtype=int32)

In [20]:
# join cluster label to the data framke
df_inserted = df_borough
df_inserted.insert(0, 'Cluster Labels', kmeans.labels_)


In [21]:
df_inserted.head()

,Cluster Labels,Neighborhood,latitude,longitude,CrimeCount
0,9,Akarana.,-36.904895,174.745517,226
1,3,Albany.,-36.727058,174.698055,829
2,6,Algies Bay.,-36.432677,174.736948,10
3,0,Ambury.,-36.949533,174.770149,51
4,0,Aorere.,-36.980988,174.832658,111


In [22]:
import pandas as pd
from sklearn import preprocessing
# scale the CrimeCount to 0 to 1

x = df_inserted[['CrimeCount']] #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_scaled = pd.DataFrame(x_scaled)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [23]:
df_inserted.insert(0, 'CrimeCountScaler', df_scaled[0])
df_inserted.head()

,CrimeCountScaler,Cluster Labels,Neighborhood,latitude,longitude,CrimeCount
0,0.098039,9,Akarana.,-36.904895,174.745517,226
1,0.360784,3,Albany.,-36.727058,174.698055,829
2,0.003922,6,Algies Bay.,-36.432677,174.736948,10
3,0.021786,0,Ambury.,-36.949533,174.770149,51
4,0.047930,0,Aorere.,-36.980988,174.832658,111


In [30]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[-36.8485,174.7633], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, scaler in zip(df_inserted['latitude'], df_inserted['longitude'], df_inserted['Neighborhood'], df_inserted['Cluster Labels'], df_inserted['CrimeCountScaler']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius= 4+10*scaler, # change the marker size here
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

Print out the head of clusters with crime count

In [25]:
for i in range(0, kclusters):
    cluster = df_inserted[df_inserted['Cluster Labels'] == i]
    print('##### {} rows in cluster {} #####'.format(cluster.shape[0], i))
    print(cluster.sample(frac=0.05))
    print('\n')

##### 92 rows in cluster 0 #####
     CrimeCountScaler  Cluster Labels   Neighborhood   latitude   longitude  \
65           0.040087               0  Favona South. -36.951370  174.807846   
46           0.197386               0     Dannemora. -36.940395  174.913945   
344          0.042702               0      Viscount. -36.969339  174.790124   
212          0.095425               0  Orakei North. -36.870288  174.814415   
141          0.082789               0       Kohuora. -36.979996  174.841432   

     CrimeCount  
65           93  
46          454  
344          99  
212         220  
141         191  


##### 12 rows in cluster 1 #####
     CrimeCountScaler  Cluster Labels        Neighborhood   latitude  \
128          0.015686               1  Kaukapakapa Rural. -36.615176   

      longitude  CrimeCount  
128  174.492121          37  


##### 61 rows in cluster 2 #####
     CrimeCountScaler  Cluster Labels        Neighborhood   latitude  \
116          0.073203               2

## 3 ETA
calling foursquare api to get the related venues for each suburbs

In [26]:
%load_ext dotenv
%dotenv

**Please set up the `.env` firstly** before running the cell below.

In [27]:
import os
import foursquare
client_id = os.environ['CLIENT_ID']
client_secret=os.environ['CLIENT_SECRET']
version=os.environ['VERSION']

In [31]:
import requests
import time
from pandas.io.json import json_normalize

# define a function to get the foursquare result
def get_venue(lat, lng, radius, limit):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        client_id, 
        client_secret, 
        version, 
        lat, 
        lng, 
        radius, 
        limit)
    results = requests.get(url).json()["response"]['groups'][0]['items']
    return results


try use the `get_venue` to get result of `albany` (-36.727058 	174.698055)

In [32]:
albany_lat_lng = (-36.727058, 174.698055)
get_venue(albany_lat_lng[0], albany_lat_lng[1], 500, 100)[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b6f9a64f964a5208df72ce3',
  'name': 'QBE Stadium',
  'location': {'address': 'Oteha Valley Road',
   'lat': -36.726937342858996,
   'lng': 174.70205945588367,
   'labeledLatLngs': [{'label': 'display',
     'lat': -36.726937342858996,
     'lng': 174.70205945588367}],
   'distance': 357,
   'postalCode': '0632',
   'cc': 'NZ',
   'neighborhood': 'North Harbour',
   'city': 'North Shore',
   'state': 'Auckland',
   'country': 'New Zealand',
   'formattedAddress': ['Oteha Valley Road',
    'North Shore 0632',
    'New Zealand']},
  'categories': [{'id': '4bf58dd8d48988d184941735',
    'name': 'Stadium',
    'pluralName': 'Stadiums',
    'shortName': 'Stadium',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/stadium_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'grou

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        results = get_venue(lat, lng, radius, limit)
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        time.sleep(1)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
all_venues = getNearbyVenues(df_inserted['Neighborhood'], df_inserted['latitude'], df_inserted['longitude'])

show up sample venues in the data frame

In [36]:
all_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akarana.,-36.904895,174.745517,Serandib Shop,-36.903338,174.742334,Grocery Store
1,Akarana.,-36.904895,174.745517,好客美食,-36.904454,174.741787,Asian Restaurant
2,Akarana.,-36.904895,174.745517,The Long's Bakehouse,-36.904080,174.741717,Bakery
3,Akarana.,-36.904895,174.745517,Kiwi Baby,-36.904939,174.741544,Kids Store
4,Akarana.,-36.904895,174.745517,The Dinning Lounge,-36.902767,174.742347,Tea Room


In [38]:
print('there are {} records of venues in all neighbourhoods'.format(all_venues.shape[0]))
print('There are {} uniques venue categories.'.format(len(all_venues['Venue Category'].unique())))

there are 3098 records of venues in all neighbourhoods
There are 219 uniques categories.


show up the venues count in each neighborhood

In [39]:
all_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Akarana.,5,5,5,5,5,5
Albany.,15,15,15,15,15,15
Algies Bay.,1,1,1,1,1,1
Ambury.,1,1,1,1,1,1
Arch Hill.,6,6,6,6,6,6
...,...,...,...,...,...,...
Whenuapai West.,4,4,4,4,4,4
Windsor Park.,5,5,5,5,5,5
Witheford.,2,2,2,2,2,2


In [45]:
num_top_venues = 5

df_v = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")
df_v['Neighborhood'] = all_venues['Neighborhood']
df_gb = df_v.groupby('Neighborhood').mean().reset_index()

for hood in df_gb['Neighborhood']:
    
    temp = df_gb[df_gb['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
#     print("-----venue category and frequency for "+hood+"----")
#     print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#     print('\n')

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

get top venue categories for neighborhoods

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_gb['Neighborhood']

for ind in np.arange(df_gb.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_gb.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akarana.,Grocery Store,Asian Restaurant,Kids Store,Tea Room,Bakery,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop
1,Albany.,Stadium,Café,Bar,Restaurant,Bubble Tea Shop,Playground,Tennis Court,Thai Restaurant,Convenience Store,Pizza Place
2,Algies Bay.,Nature Preserve,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fishing Store,Fish Market
3,Ambury.,Sandwich Place,Yoga Studio,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market,Fishing Store,Fish Market,Fish & Chips Shop
4,Arch Hill.,Café,Pizza Place,Bakery,Fish & Chips Shop,Brewery,Yoga Studio,Fast Food Restaurant,Food Court,Food & Drink Shop,Food


join venues data together

In [57]:
df_with_venues = df_inserted.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [61]:
df_with_venues.head()

,CrimeCountScaler,Cluster Labels,Neighborhood,latitude,longitude,CrimeCount,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0.098039,9,Akarana.,-36.904895,174.745517,226,Grocery Store,Asian Restaurant,Kids Store,Tea Room,Bakery,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop
1,0.360784,3,Albany.,-36.727058,174.698055,829,Stadium,Café,Bar,Restaurant,Bubble Tea Shop,Playground,Tennis Court,Thai Restaurant,Convenience Store,Pizza Place
2,0.003922,6,Algies Bay.,-36.432677,174.736948,10,Nature Preserve,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fishing Store,Fish Market
3,0.021786,0,Ambury.,-36.949533,174.770149,51,Sandwich Place,Yoga Studio,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market,Fishing Store,Fish Market,Fish & Chips Shop
4,0.047930,0,Aorere.,-36.980988,174.832658,111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


get the venues data for `Auckland Central West` with highest Crime Count rate

In [60]:
df_with_venues[ df_with_venues['Neighborhood'] == 'Auckland Central West.']

,CrimeCountScaler,Cluster Labels,Neighborhood,latitude,longitude,CrimeCount,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,1.0,9,Auckland Central West.,-36.848911,174.765226,2296,Café,Burger Joint,Vegetarian / Vegan Restaurant,Restaurant,Indian Restaurant,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Pizza Place,Lounge
